In [1]:
from tqdm import tqdm
import pandas as pd
import pbp_utils

AUTH = pbp_utils.retrieve_auth_bearer()

### PBP 2023/2024

In [2]:
# season 2023 - 2024
url_base = 'https://api-lba.procne.cloud/api/v1/'
url_calendar = 'championships/530/calendar?d='


matches_2023 = []

for day in range(1,31):
    response = pbp_utils.retrieve_info(url_base + url_calendar + str(day), auth=AUTH)
    if response:
        matches_2023.append(response['matches'])
    else:
        print("Impossibile recuperare i dati della giornata")

games = pbp_utils.retrieve_games(matches_2023)

#players_df = pd.DataFrame(
#    columns=['Team', 'Player', 'MIN', 'PTS', 'P2M', 'P2A', 'P3M', 'P3A', 'FTM', 'FTA', 'OREB', 'DREB',
#                'AST', 'TOV', 'STL', 'BLK', 'PF', 'PM'])
#team_df = pd.DataFrame(
#    columns=['Team', 'MIN', 'PTS', 'P2M', 'P2A', 'P3M', 'P3A', 'FTM', 'FTA', 'OREB', 'DREB', 'AST', 'TOV',
#                'STL', 'BLK', 'PF', 'PM'])
#opponent_df = pd.DataFrame(
#    columns=['Team', 'MIN', 'PTS', 'P2M', 'P2A', 'P3M', 'P3A', 'FTM', 'FTA', 'OREB', 'DREB', 'AST', 'TOV',
#                'STL', 'BLK', 'PF', 'PM'])
pbp_df = pd.DataFrame(
    columns=['game_id', 'data_set', 'date', 'a1', 'a2', 'a3', 'a4', 'a5', 'h1', 'h2', 'h3', 'h4', 'h5',
                'period', 'home_score', 'away_score', 'remaining_time', 'elapsed_time', 'play_length',
                'play_id', 'team', 'event_type', 'assist', 'away', 'home', 'block', 'entered', 'left', 'num',
                'opponent', 'outof', 'player', 'points', 'possession', 'reason', 'result', 'steal', 'type',
                'shot_distance', 'original_x', 'original_y', 'converted_x', 'converted_y', 'description'])

for game in tqdm(sorted(games, key=lambda game: game['game_id'])):
    game_id = game['game_id']
    if game_id not in {24095, 24114}:
        url_scores = f'championships_matches/{game_id}/scores?info=1'
        url_pbp = f'championships_matches/{game_id}/play_by_play?info=1&sort=asc'

        # retrieve starters
        response_scores = pbp_utils.retrieve_info(url_base + url_scores, auth=AUTH)
        ht_name = response_scores['match']['h_team_name']
        vt_name = response_scores['match']['v_team_name']
        starters = pbp_utils.retrieve_starters(response_scores['scores'])

        # retrieve pbp
        response_pbp = pbp_utils.retrieve_info(url_base + url_pbp, auth=AUTH)
        raw_actions = response_pbp['pbp']['actions']
        raw_actions = pbp_utils.handle_substitutions(raw_actions=raw_actions, starters=starters, game_id=game_id)
        raw_actions = pbp_utils.add_ft_count(raw_actions)
        actions = pbp_utils.create_actions(game, raw_actions)
        pbp_df = pd.concat([pbp_df, pd.DataFrame(actions)], ignore_index=True)

pbp_df.to_csv('../csvs/PBP_2023_2024.csv')

100%|██████████| 136/136 [01:01<00:00,  2.21it/s]


### PBP 2023/2024 without substitutions

In [4]:
# season 2023 - 2024
url_base = 'https://api-lba.procne.cloud/api/v1/'
url_calendar = 'championships/530/calendar?d='

matches_2023 = []

for day in range(1,31):
    response = pbp_utils.retrieve_info(url_base + url_calendar + str(day), auth=AUTH)
    if response:
        matches_2023.append(response['matches'])
    else:
        print("Impossibile recuperare i dati della giornata")

games = pbp_utils.retrieve_games(matches_2023)

#players_df = pd.DataFrame(
#    columns=['Team', 'Player', 'MIN', 'PTS', 'P2M', 'P2A', 'P3M', 'P3A', 'FTM', 'FTA', 'OREB', 'DREB',
#                'AST', 'TOV', 'STL', 'BLK', 'PF', 'PM'])
#team_df = pd.DataFrame(
#    columns=['Team', 'MIN', 'PTS', 'P2M', 'P2A', 'P3M', 'P3A', 'FTM', 'FTA', 'OREB', 'DREB', 'AST', 'TOV',
#                'STL', 'BLK', 'PF', 'PM'])
#opponent_df = pd.DataFrame(
#    columns=['Team', 'MIN', 'PTS', 'P2M', 'P2A', 'P3M', 'P3A', 'FTM', 'FTA', 'OREB', 'DREB', 'AST', 'TOV',
#                'STL', 'BLK', 'PF', 'PM'])
pbp_df = pd.DataFrame(
    columns=['game_id', 'data_set', 'date', 'a1', 'a2', 'a3', 'a4', 'a5', 'h1', 'h2', 'h3', 'h4', 'h5',
                'period', 'home_score', 'away_score', 'remaining_time', 'elapsed_time', 'play_length',
                'play_id', 'team', 'event_type', 'assist', 'away', 'home', 'block', 'entered', 'left', 'num',
                'opponent', 'outof', 'player', 'points', 'possession', 'reason', 'result', 'steal', 'type',
                'shot_distance', 'original_x', 'original_y', 'converted_x', 'converted_y', 'description'])

for game in tqdm(sorted(games, key=lambda game: game['game_id'])):
    game_id = game['game_id']

    url_scores = f'championships_matches/{game_id}/scores?info=1'
    url_pbp = f'championships_matches/{game_id}/play_by_play?info=1&sort=asc'

    # retrieve starters
    response_scores = pbp_utils.retrieve_info(url_base + url_scores, auth=AUTH)
    ht_name = response_scores['match']['h_team_name']
    vt_name = response_scores['match']['v_team_name']
    starters = pbp_utils.retrieve_starters(response_scores['scores'])

    # retrieve pbp
    response_pbp = pbp_utils.retrieve_info(url_base + url_pbp, auth=AUTH)
    raw_actions = response_pbp['pbp']['actions']
    raw_actions = pbp_utils.remove_substitutions(raw_actions)
    raw_actions = pbp_utils.add_ft_count(raw_actions)
    actions = pbp_utils.create_actions(game, raw_actions , substitutions=False)
    pbp_df = pd.concat([pbp_df, pd.DataFrame(actions)], ignore_index=True)

pbp_df.to_csv('../csvs/PBP_2023_2024_without_substitutions.csv')


100%|██████████| 136/136 [00:58<00:00,  2.33it/s]
